In [1]:
import os
import sys
os.chdir("./..")
sys.path.append(os.getcwd())

import yaml

import albumentations as A
import cv2

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from utils.paths import REPO_PATH
from utils.plots import visualize_bbox, visualize

In [2]:
# Plot a line graph
plt.plot([1.5, 3.0])

# Plot the title, X and Y axis labels
plt.title("Non Interactive Mode")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")

Text(0, 0.5, 'Y-axis')

In [3]:
TEST_IMAGE_PATH = f"{REPO_PATH}/data/full_dataset/images/IMG_7288_0038.jpg"
TEST_LABEL_PATH = f"{REPO_PATH}/data/full_dataset/labels/IMG_7288_0038.txt"
INHALER_META_PATH = f"{REPO_PATH}/data/inhaler.yaml"

In [4]:
with open(INHALER_META_PATH, 'r') as file:
    category_id_to_name = yaml.safe_load(file)['names']

In [5]:
test_image = cv2.imread(TEST_IMAGE_PATH)
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
with open(TEST_LABEL_PATH, "r") as f:
    test_label = f.read()
    test_label = test_label.replace("\n", "").split(" ")
    class_id = int(test_label[0])
    bbox = [float(x) for x in test_label[1:]]
    test_label = [*bbox, class_id]

plt.imshow(test_image)
plt.show()

C:\Users\ernes\AppData\Local\Temp\ipykernel_12544\2705041863.py:11: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [5]:
transform = A.Compose([
    A.RandomSizedBBoxSafeCrop(width=test_image.shape[1], height=test_image.shape[0], erosion_rate=1),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.5, contrast_limit=0.5, p=0.7),
], bbox_params=A.BboxParams(format='coco', min_visibility=0.7))

In [6]:
transformed = transform(image=test_image, bboxes=[test_label])
transformed_image = transformed['image']
transformed_bboxes = transformed['bboxes']

bboxes = [t[:-1] for t in transformed_bboxes]
category_ids = [t[-1] for t in transformed_bboxes]

# visualize(transformed_image, bboxes, category_ids, category_id_to_name)

In [8]:
plt.imshow(transformed_image)